## Corporate Valuation: Discounted Cash Flow Approach

In [1]:
'''Create a class which determines the following:
        - the Weighted Average Cost of Capital (WACC) and calculation of the enterprise value 
        - the cost of capital (for Total Cash Flow Approach) and calculation of the enterprise value
'''



class DCF_Approach():
    
    def __init__(self, cashflows):
        self.cashflows = cashflows
    
    
    
    # WACC calculation
    def wacc(self, equity, liabilities, beta_levered, i_r, s_ci=0.14, s_ct=0.15, mrp=0.05, risk_premium=0.02):
        
        # i_r: risk-free base rate; mrp: market risk premium; r_liab: borrowing costs
        # s_ci: commercial income tax rate; s_ct: corporate tax rate
                             
        equity_cost = i_r + beta_levered * mrp    
        tax_shield = (1 - 0.75 * s_ci) * (1 - s_ct)                                  
        r_liab = i_r + risk_premium
        liability_cost = r_liab * tax_shield                          
        cost_of_capital = equity_cost * (equity/(equity+liabilities)) + liability_cost * (liabilities/(equity+liabilities))
        return cost_of_capital
    
    
    
    
    # Define the enterprise value using the DCF/WACC technique (with wacc as discount rate)
    def valuation_wacc(self, equity, liabilities, beta_levered=0.96, i_r=0.02,  growth_rate=0.05):
        
        cost_of_capital = self.wacc(equity, liabilities, beta_levered, i_r)
        
        t = 1
        total_value = 0
        for fcf in self.cashflows:
            discounted_cashflow = fcf / ((1 + cost_of_capital)**t)
            total_value = (total_value + discounted_cashflow)
            t += 1
            #print(discounted_cashflow)
        
        dcf_before_tv = total_value
        #print("Corporate Value before Terminal Value: ", str(dcf_before_tv))
        
        T = len(self.cashflows)
        tv = ((self.cashflows[-1]*(1 + growth_rate)) / (cost_of_capital - growth_rate)) / ((1 + cost_of_capital)**T)
        #print("Terminal Value: ", str(tv))
    
        corporate_value = (dcf_before_tv + tv)
        #print("Corporate Value: ", str(corporate_value))
        
        return corporate_value.astype(np.int32)


    
    

    # Discount rate for the Total Cash Flow Approach
    def cost_of_capital_tcf(self, equity, liabilities, beta_levered, i_r, mrp=0.05, risk_premium=0.02):
        
        equity_cost = i_r + beta_levered * mrp                                   
        liability_cost = i_r + risk_premium                      
        cost_of_capital = equity_cost * (equity/(equity+liabilities)) + liability_cost * (liabilities/(equity+liabilities))
        return cost_of_capital


    
    
    # Define the enterprise value using the TCF technique (with cost_of_capital_tcf as discount rate)
    def valuation_tcf(self, equity, liabilities, beta_levered=0.96, i_r=0.02,  growth_rate=0.05):
        
        cost_of_capital = self.cost_of_capital_tcf(equity, liabilities, beta_levered, i_r)
        
        t = 1
        total_value = 0
        for tcf in self.cashflows:
            discounted_cashflow = tcf / ((1 + cost_of_capital)**t)
            total_value = total_value + discounted_cashflow
            t += 1
            #print(discounted_cashflow)
        
        dcf_before_tv = total_value
        #print("Corporate Value before Terminal Value: ", str(dcf_before_tv))
        
        T = len(self.cashflows)
        tv = ((self.cashflows[-1]*(1 + growth_rate)) / (cost_of_capital - growth_rate)) / ((1 + cost_of_capital)**T)
        #print("Terminal Value: ", str(tv))
    
        corporate_value = dcf_before_tv + tv - liabilities
        #print("Corporate Value: ", str(corporate_value))
        
        return corporate_value.astype(np.int32)

In [2]:
# Example Case

import pandas as pd
import numpy as np


# Create a DataFrame with the needed figures (index/rows) and the planning period (columns, with past and future).
ind = ["EBIT", "Gross Cash Flow", "Free Cash Flow", "Total Cash Flow"]
cols = ["2018", "2019", "2020", "2021", "2022", "2023", "2024", "2025"]

data = pd.DataFrame(index=ind, columns=cols) 

data["2018"] = [300, 250, 220, 210]
data["2019"] = [310, 280, 240, 235]
data["2020"] = [315, 290, 250, 241]
data["2021"] = [311, 275, 235, 228]
data["2022"] = [318, 296, 253, 243]
data["2023"] = [330, 303, 288, 269]
data["2024"] = [342, 311, 299, 281]
data["2025"] = [360, 330, 311, 300]
data

,2018,2019,2020,2021,2022,2023,2024,2025
EBIT,300,310,315,311,318,330,342,360
Gross Cash Flow,250,280,290,275,296,303,311,330
Free Cash Flow,220,240,250,235,253,288,299,311
Total Cash Flow,210,235,241,228,243,269,281,300


In [3]:
# Create a list for saving the future and forecasted Free Cash Flows.

FreeCashFlows = []                  # empty list

for fcf in data.loc["Free Cash Flow", ["2021","2022","2023","2024","2025"]].values:          # 2021-2025: forecast
    FreeCashFlows.append(fcf)

print("Free Cash Flows:")
print(FreeCashFlows)                # list with numbers 
print()



# Calculate and get the WACC and enterprise value.

dcf = DCF_Approach(FreeCashFlows)                      # object

wacc = round(dcf.wacc(180, 90, 0.96, 0.02), 4)*100
print("Weighted Average Cost of Capital:\t" + str(wacc) + " %")

corporate_value = dcf.valuation_wacc(180, 90)
print("Enterprise Value:\t\t\t" + str(corporate_value) + " €")

Free Cash Flows:
[235, 253, 288, 299, 311]

Weighted Average Cost of Capital:	5.55 %
Enterprise Value:			46691 €


In [4]:
# Create a list for saving the future and forecasted Total Cash Flows.

TotalCashFlows = []

for tcf in data.loc["Total Cash Flow", ["2021","2022","2023","2024","2025"]].values:          # 2021-2025: forecast
    TotalCashFlows.append(tcf)

print("Total Cashflows:")
print(TotalCashFlows)
print()


# Calculate and get the cost of capital and enterprise value.

dcf1 = DCF_Approach(TotalCashFlows) 

discount_rate = round(dcf1.cost_of_capital_tcf(180, 90, 0.96, 0.02), 4)*100
print("Cost of Capital for TCF-Approach:\t" + str(discount_rate) + " %")

business_value = dcf1.valuation_tcf(180, 90)
print("Enterprise Value:\t\t\t" + str(business_value) + " €")


Total Cashflows:
[228, 243, 269, 281, 300]

Cost of Capital for TCF-Approach:	5.87 %
Enterprise Value:			28349 €
